In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import random
#LIBOR = .025

In [2]:
#loan charateristics

#face value
FV = 1000000

#annual rate
rate = 0.04

#compounding periods
term = 10

#coupon
coupon = 0.03*FV

PD = 0.02

RR = 0.75

In [3]:

PV = np.round(np.abs((npf.pv(rate, term, coupon, FV, when='end'))),2)
print(f'The present value of the risk free bond is {PV} USD')

The present value of the risk free bond is 918891.04 USD


In [4]:

YTM = ((coupon + (FV-PV)/ term)) / ((FV + PV)/2)
print(f'The YTM for this bond is {np.round(YTM,5) *100}%')

The YTM for this bond is 3.972%


In [5]:

YTM2 = np.power((np.power((1+YTM),term))/(PD*RR+(1 - PD)),(1/term))-1
print(f'The Risky YTM for this bond is {np.round(YTM2,5) *100}%')

The Risky YTM for this bond is 4.024%


In [29]:
class Loan:

    #integer, float, datetime
    #treasury rate(annual) coupon_rate(annual)
    def __init__(self, size, treasury_rate, interest_rate, maturity):
        self.size = size
        self.t_rate = treasury_rate
        self.rate = interest_rate
        self.maturity = maturity
        self.defaulted = False
        
    def PV(self, start_date):
        
        term = (self.maturity.year - start_date.year)*12 + (self.maturity.month - start_date.month)
        interest = self.size * (self.rate) / 12
        t_rate = self.t_rate/12
        PV = np.round(np.abs((npf.pv(t_rate, term, rate, self.size, when='end'))),2)
        self.pv = PV
        self.term = term
        print(f'The present value of the risk free bond is {PV} USD')
     
    def YTM(self):
        
        YTM = ((self.rate + (self.size - self.pv)/ self.term)) / ((self.size + self.pv)/2)
        self.ytm = YTM
        print(f'The monthly yield to maturity for this bond is {np.round(YTM,5) *100}%')
                      

    #creates a list of dictionaries which represents a loan
    def gen_cash_flow(self, start_date, default_prob, recovery_rate):
        self.cash_flows = []
        default_prob = (1-(1-default_prob)**(1/12))
        while len(self.cash_flows)==0 or self.cash_flows[-1]['size']>0:
            self.cash_flows.append({})
            self.cash_flows[-1]['date'] = start_date
            start_date = start_date + relativedelta(months = 1)
            #self.cashflows[-1]['month'] = 0
  
            
            self.cash_flows[-1]['size'] = self.size
            if self.defaulted:
                self.cash_flows[-1]['interest'] = 0
                self.cash_flows[-1]['principal'] = 0
                self.cash_flows[-1]['loss'] = self.size * (1-recovery_rate)
                self.cash_flows[-1]['recovery'] = self.size * recovery_rate
                self.cash_flows[-1]['size'] = 0
            else:
                interest = self.size * (self.rate) / 12
                #discounted interest = intreset 
                self.cash_flows[-1]['interest'] = interest
                self.cash_flows[-1]['loss'] = 0
                self.cash_flows[-1]['recovery'] = 0

                if start_date > self.maturity:
                    self.cash_flows[-1]['principal'] = self.size
                    self.cash_flows[-1]['size'] = 0
                else:
                    self.cash_flows[-1]['principal'] = 0

            if random.random() < default_prob:
                self.defaulted = True
                
    def cf_table(self):
        df = pd.DataFrame(self.cash_flows).set_index('date')
        return df       

In [17]:
loan = Loan(10000, 0.04, 0.03, datetime.date(2022, 1, 1))
loan.gen_cash_flow(start_date = datetime.date(2020, 1, 1), default_prob = 0 , recovery_rate = 0.5)

In [8]:
loan.PV(datetime.date(2020, 1, 1))
loan.YTM()

The present value of the risk free bond is 9233.31 USD
The monthly yield to maturity for this bond is 0.333%


start_date = datetime.date(2021, 1, 1)
maturity = start_date + relativedelta(months = 1)
maturity.strftime("%m/%d/%Y")

In [30]:
loan.cf_table()

,size,interest,loss,recovery,principal
date,,,,,
2020-01-01,10000,25.0,0,0,0
2020-02-01,10000,25.0,0,0,0
2020-03-01,10000,25.0,0,0,0
2020-04-01,10000,25.0,0,0,0
2020-05-01,10000,25.0,0,0,0
2020-06-01,10000,25.0,0,0,0
2020-07-01,10000,25.0,0,0,0
2020-08-01,10000,25.0,0,0,0
2020-09-01,10000,25.0,0,0,0
